In [1]:
import numpy as np

In [2]:
def get_model(save_name='ResNet50.hdh5', image_shape=(224, 224, 3)):
    import os
    new_model = not os.path.exists(save_name)
    if new_model:
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(
            include_top=False, weights='imagenet',
            input_tensor=None, input_shape=image_shape, pooling='max'
        )
        model.save(save_name)
    else:
        from keras.models import load_model
        model = load_model(save_name)
    return model, new_model

In [3]:
initial_model, new_model = get_model()
if new_model:
    print('new model')
else:
    print('old model')

Using TensorFlow backend.


old model


/usr/lib/python3.6/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def get_data(image_shape=(224, 224, 3)):
    from skimage.transform import resize
    from skimage.io import imread
    import os
    images = []
    labels = []
    with open('public_data/00_input/train/gt.csv') as navigate_file:
        for i, line in enumerate(navigate_file):
            if i == 0:
                continue
            image_name, label = line.split(',')                
            images.append(resize(
                imread(os.path.join(
                    'public_data/00_input/train/images/',
                    image_name
                )),
                image_shape,
                mode='reflect'
            ))
            labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
x, y = get_data()

In [6]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [7]:
num_classes = len(set(y))
y = np.eye(num_classes)[y]
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

In [9]:
regularization_lambda = 0.0005

last = initial_model.output

prediction = Dense(
    num_classes, activation='softmax',
    kernel_regularizer=regularizers.l2(regularization_lambda),
    bias_regularizer=regularizers.l2(regularization_lambda)
)(last)

model = Model(initial_model.input, prediction)

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Nadam(lr=0.00005),
    metrics=['categorical_accuracy']
)

In [ ]:
batch_size = 16
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=300,
    validation_data=(test_x, test_y)
)

Epoch 1/300
109/109 [==============================] - 85s - loss: 3.1477 - categorical_accuracy: 0.2374 - val_loss: 4.2747 - val_categorical_accuracy: 0.0160
Epoch 2/300
109/109 [==============================] - 76s - loss: 1.3168 - categorical_accuracy: 0.6996 - val_loss: 4.2211 - val_categorical_accuracy: 0.0213
Epoch 3/300
109/109 [==============================] - 76s - loss: 0.7486 - categorical_accuracy: 0.8358 - val_loss: 4.1156 - val_categorical_accuracy: 0.0173
Epoch 4/300
109/109 [==============================] - 76s - loss: 0.4573 - categorical_accuracy: 0.9249 - val_loss: 4.0102 - val_categorical_accuracy: 0.0320
Epoch 5/300
109/109 [==============================] - 76s - loss: 0.3214 - categorical_accuracy: 0.9438 - val_loss: 3.8108 - val_categorical_accuracy: 0.0733
Epoch 6/300
109/109 [==============================] - 76s - loss: 0.2359 - categorical_accuracy: 0.9622 - val_loss: 3.3903 - val_categorical_accuracy: 0.1667
Epoch 7/300
109/109 [=========================

106/109 [============================>.] - ETA: 1s - loss: 0.1071 - categorical_accuracy: 0.9849

In [12]:
model.save('0.72_score_model.hdh5')